In [19]:
import toy_data.sine_wave as swg
import numpy as np
import bokeh.io
import bokeh.plotting as bp
import bokeh.charts as bc
import tensorflow as tf
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
sw = swg.SineWave()
swg.visualize_1D_regression(sw)

In [3]:
d_x = sw.y[sw.X.argsort(axis=0)]
d_x.shape

(200, 1, 1)

In [4]:
d_X = d_x[:,:,0][np.newaxis, :]
x_dim = 1
d_X.shape # No. of samples, Time, No. of dimensions

(1, 200, 1)

In [5]:
batch_size = 1

In [6]:
ef = lambda x: tf.expand_dims(x, -1) # expand forward
eb = lambda x: tf.expand_dims(x, 0) # expand backward

In [7]:
state_size = 10
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [batch_size, None, x_dim]) # [batch_size, num_steps]
x_in = x[:,:-1, :]
x_out = x[:, 1:, :]
cell = tf.contrib.rnn.BasicLSTMCell(state_size)
rnn_out, final_state = tf.nn.dynamic_rnn(cell, x_in, dtype=tf.float32)

W = tf.get_variable('output_embedding', (state_size, x_dim), initializer=tf.truncated_normal_initializer())
out_pred = tf.reduce_sum(ef(rnn_out) * eb(eb(W)), axis=-2)

loss_all = tf.square(out_pred - x_out)
loss = tf.reduce_sum(loss_all)
train_step = tf.train.RMSPropOptimizer(0.01).minimize(loss)

In [11]:
inputs = {x: d_X}

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [13]:
for i in range(50):
    sess.run(train_step, feed_dict=inputs)

In [16]:
print(sess.run(loss, feed_dict=inputs))
np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
val_actual = sess.run(x_out, feed_dict=inputs).ravel()
val_pred = sess.run(out_pred, feed_dict=inputs).ravel()
print('Actual value\n', val_actual)
print('Predictions\n', val_pred)

217.61
Actual value
 [ 1.73  1.81 -1.24  1.18  1.78  2.62  0.74  1.86  2.13  1.50  2.59  3.14
  2.59  3.71  5.44  1.19  2.79  3.37  3.38  1.80  1.65  3.87  4.15  3.03
  2.58  2.66  3.94  0.52  2.79  0.39  0.26  1.11  1.75  0.24 -0.02 -0.47
 -1.96 -0.97 -1.53 -2.92 -1.59 -0.60 -1.40 -0.98 -1.47 -1.53 -3.48 -3.89
 -5.77 -2.65 -1.44 -4.13 -3.13 -3.08 -4.55 -3.49 -2.89 -3.48 -2.10 -0.63
 -2.31 -0.72 -1.96  0.41 -1.81 -0.34 -0.01 -2.74 -1.12 -1.12  0.36  1.53
  1.91 -0.05  2.69  1.54  0.94  1.63  4.03  2.15  2.34  2.84  4.21  2.33
  2.58  2.47  4.30  3.14  4.35  2.19  1.46  0.16  1.22  0.57  0.21  1.15
 -1.23 -1.79 -3.91 -2.74 -2.07 -3.23 -2.12 -4.40 -1.93 -1.79 -0.76 -2.85
 -3.99 -1.06 -1.82 -1.63  0.53  0.97  0.05  0.74  1.43  0.15  0.92  1.61
  2.88  2.02  2.38  3.35  2.21  2.16  3.71  1.69  2.37  3.08  3.34  3.63
  2.75  2.49  2.74  2.17  2.97  2.38  3.55  2.27  0.68  1.05  0.51  1.42
  1.09  1.21  0.08  0.03  1.44 -0.35 -0.47 -0.52 -3.57 -1.73 -1.97 -2.44
 -2.10 -4.95 -1.68 -3.34 -4.39

In [23]:
p = bc.Line([val_actual, val_pred])
bc.show(p)